In [1]:
import branca.colormap as cm
import folium
import geopandas as gpd
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()
import json

pd.set_option("display.max_columns", 100)

## Geographical data

In [2]:
# GeoJson Shapefile
sectors_geo = r"../data/maps/stat_sectors/json/RBC_stat_sectors_2021.json"

# open the json 
with open(sectors_geo) as sectors_file:
    sectors_json = json.load(sectors_file)
    
# Extract names of each statistical sector in the city of Brussels (RBC)
names_json = []
for index in range(len(sectors_json["features"])):
    names_json.append(sectors_json["features"][index]["properties"]["Sector"])

## Values for Choropleth Map

In [3]:
population = pd.read_csv("../data/population/population_stats_sectors.csv")
population = population[["CD_SECTOR", "POPULATION"]]
population.rename(
    columns={"CD_SECTOR": "Stat_Sector", "POPULATION": "Population"}, inplace=True
)
population.head(5)

,Stat_Sector,Population
0,11001A00-,2490
1,11001A01-,2755
2,11001A020,732
3,11001A030,431
4,11001A042,351


In [4]:
# Adding the population value to each sector of the geojson file
for index in tqdm(range(len(sectors_json["features"]))):

    temp_dict = sectors_json["features"][index]["properties"]
    temp_dict["Population"] = "Population: " + str(
        population[population["Stat_Sector"] == temp_dict["Sector"]]["Population"].item()
    )
    sectors_json["features"][index]["properties"] = temp_dict

  0%|          | 0/724 [00:00<?, ?it/s]

## Generate Choropleth Map

In [5]:
# initial map coordinates
coords = [50.84892175574389, 4.3514911042124345]

world = folium.Map(
    location=[coords[0], coords[1]], zoom_start=12.45, tiles="openstreetmap"
)

# add tile layers to the map
tiles = ["stamenwatercolor", "cartodbpositron", "openstreetmap", "stamenterrain"]
for tile in tiles:
    folium.TileLayer(tile).add_to(world)

choropleth = folium.Choropleth(
    geo_data=sectors_json,
    data=population,
    columns=["Stat_Sector", "Population"],
    # threshold_scale=threshold_scale,
    key_on="feature.properties.Sector",
    fill_color="YlOrRd",
    fill_opacity=0.6,
    line_opacity=0.25,
    legend_name="Population in Region-Brussels-Capital",
    highlight=True,
    smooth_factor=0,
).add_to(world)

style_function = "font-size: 15px"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(
        ["Sector", "Population"], style=style_function, labels=False
    )
)

# create a layer control
folium.LayerControl().add_to(world)

world